<center> <h1>Widgets in C++ with xeus and cling</h1> </center>

## Button widget

In [1]:
#include <iostream>
#include "xwidgets/xbutton.hpp"

In [2]:
xw::button button;

In [3]:
void foo()
{
    button.description = "button";
}

In [4]:
button.on_click(foo);
    
button.display()

A Jupyter widget

## Numerical input

In [5]:
#include "xwidgets/xnumeral.hpp"

In [6]:
xw::numeral<double> numeral;
numeral.display()

A Jupyter widget

In [7]:
numeral.value = 4;

In [8]:
numeral.value()

4

## Image

In [7]:
#include "xwidgets/ximage.hpp"
#include "xwidgets/xprogress.hpp"

In [8]:
auto input = xw::image_from_file("./images/my_awesome_cat.jpg").finalize();
input

A Jupyter widget

<center> <h1>C++ backend for the jupyter-leaflet map visualization library</h1> </center>

In [9]:
#include <random>
#include <array>
#include <vector>
#include <chrono>
#include <thread>

#include "xwidgets/xlink.hpp"
#include "xwidgets/xslider.hpp"

#include "xleaflet/xmap.hpp"
#include "xleaflet/xbasemaps.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xheatmap.hpp"

# Base example with xwidgets

In [10]:
std::array<double, 2> center = {52.204793, 360.121558};

auto map1 = xlf::map_generator()
    .center(center)
    .zoom(9)
    .finalize();
map1.display();

A Jupyter widget

## Link the zoom level to a slider widget

In [ ]:
auto slider = xw::slider_generator<int>()
    .max(map1.max_zoom)
    .min(map1.min_zoom)
    .description("zoom:")
    .value(map1.zoom)
    .finalize();

auto link = xw::link(slider, "value", map1, "zoom");

slider

## Add a Strava layer

In [ ]:
auto strava = xlf::basemap({"Strava", "All"});

map1.add_layer(strava);

## Add a marker

In [ ]:
auto marker = xlf::marker_generator()
    .location({52.1, 359.9})
    .finalize();

map1.add_layer(marker);

# Heatmap layer

In [ ]:
auto map2 = xlf::map_generator()
    .center({37.58, 261.65})
    .zoom(5)
    .finalize();
map2

In [ ]:
std::random_device rd;
std::mt19937 mt(rd());
std::uniform_real_distribution<double> rd_latitude(34.0, 40.0);
std::uniform_real_distribution<double> rd_longitude(255.0, 265.0);
std::uniform_real_distribution<double> rd_intensity(0.0, 1000.0);

std::vector<std::array<double, 3>> heatmap_points;

for (std::size_t i = 0; i < 100; ++i)
{
    heatmap_points.push_back({rd_latitude(mt), rd_longitude(mt), rd_intensity(mt)});
}

In [ ]:
auto heatmap = xlf::heatmap_generator()
    .locations(heatmap_points)
    .finalize();
map2.add_layer(heatmap);

In [ ]:
for (std::size_t i = 0; i < 10; ++i)
{
    std::this_thread::sleep_for(std::chrono::seconds(1));
    heatmap.radius = heatmap.radius() + 1;
}

## Velocity layer

In [ ]:
#include <fstream>
#include "nlohmann/json.hpp"
#include "xleaflet/xvelocity.hpp"

In [ ]:
auto map3 = xlf::map_generator()
    .center({37.58, 261.65})
    .zoom(5)
    .finalize();

auto base_layer = xlf::basemap({"CartoDB", "DarkMatter"});
map3.add_layer(base_layer);

map3

In [ ]:
std::ifstream file("velocity_data.json");
nlohmann::json data;
file >> data;

auto velocity = xlf::velocity_generator()
    .data(data)
    .velocity_scale(0.01)
    .max_velocity(20)
    .display_options(R"({
        "velocityType": "Global Wind",
        "displayPosition": "bottomleft",
        "displayEmptyString": "No wind data"
    })")
    .finalize();

map3.add_layer(velocity);